In [1]:
import os
import cv2
import glob
import pickle
import argparse
import numpy as np
from src.utils import *
from src.yolo_net import Yolo

In [2]:
image_path = 'testImage.JPG'
CLASSES = ['aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus', 'car', 'cat', 'chair', 'cow',
           'diningtable', 'dog', 'horse', 'motorbike', 'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tvmonitor']
image_size = 448

In [3]:
model = Yolo(20)
model.cuda()
model.load_state_dict(torch.load('trained_models/yolov2_voc_weights1.pth'))
model.eval()
colors = pickle.load(open("src/pallete", "rb"))

image = cv2.imread(image_path)
image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
height, width = image.shape[:2]
image = cv2.resize(image, (image_size, image_size))
image = np.transpose(np.array(image, dtype=np.float32), (2, 0, 1))
image = image[None, :, :, :]
width_ratio = float(image_size) / width
height_ratio = float(image_size) / height
data = Variable(torch.FloatTensor(image))
if torch.cuda.is_available():
    data = data.cuda()
with torch.no_grad():
    logits = model(data)
    predictions = post_processing(logits, image_size, CLASSES, model.anchors, 0.35, 0.5)

if len(predictions) != 0:
    predictions = predictions[0]
    output_image = cv2.imread(image_path)
    for pred in predictions:
        xmin = int(max(pred[0] / width_ratio, 0))
        ymin = int(max(pred[1] / height_ratio, 0))
        xmax = int(min((pred[0] + pred[2]) / width_ratio, width))
        ymax = int(min((pred[1] + pred[3]) / height_ratio, height))
        color = colors[CLASSES.index(pred[5])]
        cv2.rectangle(output_image, (xmin, ymin), (xmax, ymax), color, 2)
        text_size = cv2.getTextSize(pred[5] + ' : %.2f' % pred[4], cv2.FONT_HERSHEY_SIMPLEX, 1, 1)[0]
        cv2.rectangle(output_image, (xmin, ymin), (xmin + text_size[0] + 3, ymin + text_size[1] + 4), color, -1)
        cv2.putText(
            output_image, pred[5] + ' : %.2f' % pred[4],
            (xmin, ymin + text_size[1] + 4), cv2.FONT_HERSHEY_SIMPLEX, 1,
            (255, 255, 255), 1)
        print("Object: {}, Bounding box: ({},{}) ({},{})".format(pred[5], xmin, xmax, ymin, ymax))
    cv2.imwrite(image_path.split('.')[0] + '_result.' + image_path.split('.')[1], output_image)

Object: cat, Bounding box: (330,875) (217,786)
